In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

OUTPUT = 1

data = pd.read_csv("breast-cancer.csv")
diagnosis = data['diagnosis']

In [ ]:
if OUTPUT:
    data.head()
    data.info()
    print(data.describe())
    categories = ['M', 'L']
    plt.figure()
    class_count = diagnosis.value_counts()
    sns.barplot(x=class_count.index, y=class_count.values)

# Data Preprocessing

1. Check if there are any empty values

In [ ]:
if OUTPUT:
    data.isnull().any()

2. Check for Outliers

In [ ]:
plt.figure(figsize=(15, 11))
features = data.iloc[:, 2:]
for i in range(len(features.columns)):
    plt.subplot(5, 6, i+1)
    sns.boxplot(y=features.columns[i], data=features)
plt.tight_layout
plt.show()

In [ ]:
plt.figure(figsize=(25,20))
sns.heatmap(data=(data.iloc[:, 2:]).corr(), linewidths=5, cmap='crest')

In [ ]:
## Outlier Detection

### All entries

# index = []

# length = 0

# features = list(data)
# print(features)
# features = features[2:]

# for feature in features:
#     values = data[feature].values

#     q1 = np.percentile(values, 25)
#     q2 = np.percentile(values, 50)
#     q3 = np.percentile(values, 75)
#     iqr = q3 - q1
    
#     max = q3 + (1.5 * iqr)
#     min = q1 - (1.5 * iqr)
#     count = 0
#     for i in range(len(values)):
#         if values[i] > max or values[i] < min:
#             count += 1
#             index.append(i)
#     print(f"For {feature} there are {count} outliers")
      
      
# index = list(dict.fromkeys(index))

# print(f"Number of entries in old dataframe: {len(data)}")

# no_outliers_data = data.drop(index=index)      
# print(f"Number of outliers in new dataframe: {len(no_outliers_data)}")

# categories = ['M', 'L']
# plt.figure()
# class_count = no_outliers_data['diagnosis'].value_counts()
# sns.barplot(x=class_count.index, y=class_count.values)



## Outlier Detection for malignant and benign alone 

In [ ]:

benign = data[data['diagnosis'] == 'B'].reset_index(drop=True)

index = []
features = list(benign)
features = features[2:]
for feature in features:
    values = benign[feature].values

    q1 = np.percentile(values, 25)
    q2 = np.percentile(values, 50)
    q3 = np.percentile(values, 75)
    iqr = q3 - q1
    
    max = q3 + (1.5 * iqr)
    min = q1 - (1.5 * iqr)
    count = 0
    for i in range(len(values)):
        if values[i] > max or values[i] < min:
            count += 1
            index.append(i)


index = list(dict.fromkeys(index))
print(len(index))


103


In [34]:
malignant = data[data['diagnosis'] == 'M'].reset_index(drop=True)


index = []
features = list(malignant)
features = features[2:]
for feature in features:
    values = malignant[feature].values

    q1 = np.percentile(values, 25)
    q2 = np.percentile(values, 50)
    q3 = np.percentile(values, 75)
    iqr = q3 - q1
    
    max = q3 + (1.5 * iqr)
    min = q1 - (1.5 * iqr)
    count = 0
    for i in range(len(values)):
        if values[i] > max or values[i] < min:
            count += 1
            index.append(i)


index = list(dict.fromkeys(index))
print(len(index))

61
